<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP6_LCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LCA- Topic Modeling
#Objective
1. Get books from https://www.gutenberg.org 
2. Clean content from book data
3. Get topics using LCA
4. Get content from Reddit
5. Clean content from thread data
6. Get topics using LCA

In [7]:
import urllib
from bs4 import BeautifulSoup

from urllib.request import urlopen
from bs4 import BeautifulSoup
documents = []

import re
regex = re.compile(r'<[^>]+>')

def remove_html(string):
    return regex.sub('', string)

def getContent(url): 
  #url = "https://www.gutenberg.org/files/55/55-h/55-h.htm" 
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")
  for script in soup(["script", "style"]):
    script.extract()
  text = soup.get_text()
  documents.append(remove_html(text))

url="https://www.gutenberg.org/files/55/55-h/55-h.htm"
getContent(url)
(documents[0])[:500]
url = "https://www.gutenberg.org/files/54/54-h/54-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/33361/33361-h/33361-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/22566/22566-h/22566-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/26624/26624-h/26624-h.htm" 
getContent(url)



In [8]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

tfidf_vect = CountVectorizer(analyzer='word', stop_words = 'english')
df=tfidf_vect.fit_transform(documents)
vocab = tfidf_vect.get_feature_names_out()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
vocab

array(['00', '000', '10', ..., 'zoroaster', 'zuz', 'zy'], dtype=object)

In [ ]:
print(df.shape)

(5, 8851)


In [9]:
print(df[0])

  (0, 5966)	89
  (0, 3695)	99
  (0, 2645)	14
  (0, 8716)	28
  (0, 8697)	44
  (0, 5411)	169
  (0, 3303)	5
  (0, 860)	5
  (0, 8323)	28
  (0, 8255)	15
  (0, 7421)	19
  (0, 5481)	2
  (0, 8749)	19
  (0, 1915)	4
  (0, 6420)	2
  (0, 8573)	2
  (0, 1886)	12
  (0, 766)	71
  (0, 7827)	22
  (0, 4567)	18
  (0, 4108)	3
  (0, 5320)	4
  (0, 8794)	10
  (0, 5353)	10
  (0, 4644)	7
  :	:
  (0, 5427)	1
  (0, 2484)	1
  (0, 4910)	1
  (0, 401)	1
  (0, 8527)	1
  (0, 2005)	1
  (0, 1352)	1
  (0, 5957)	1
  (0, 4913)	1
  (0, 3775)	1
  (0, 5361)	1
  (0, 6889)	1
  (0, 5953)	1
  (0, 4675)	1
  (0, 5168)	1
  (0, 8434)	1
  (0, 1783)	1
  (0, 5136)	1
  (0, 2655)	1
  (0, 4749)	1
  (0, 5626)	1
  (0, 2958)	1
  (0, 5952)	1
  (0, 7601)	1
  (0, 5177)	1


In [ ]:
print (vocab[5177])

newsletter


In [10]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, doc_topic_prior=1)
lda.fit(df)
lda.components_[0].shape

(8851,)

In [11]:
import numpy as np 
topic_words = {}
n_top_words = 20
for topic, comp in enumerate(lda.components_):
    # print(topic, comp)
    word_idx = np.argsort(comp)[::-1][:n_top_words] #argsort to get index, and [::-1] to sort in descending
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
    # break
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework
Topic: 1
  dorothy, said, little, man, pg, oz, asked, scarecrow, gutenberg, king, girl, wizard, project, ozma, tin, good, woodman, great, shaggy, head
Topic: 2
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework
Topic: 3
  said, tip, scarecrow, horse, jack, saw, woodman, tin, pumpkinhead, mombi, city, boy, bug, woggle, head, gutenberg, glinda, good, old, project
Topic: 4
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework


#Reddit Data

In [12]:
# !pip install praw
!pip install asyncpraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# import praw
import asyncpraw as praw
from asyncpraw.models import MoreComments
reddit = praw.Reddit(
    user_agent="Comment Extraction (by u/Bitter_Researcher859)",
    client_id="5YUVEJjNjYgNfwrR0Jle_Q",
    client_secret="y1Yw5hcMa874BIzhbG5DSFnE9f_ncg",
    redirect_uri='http://127.0.0.1:65010/'
                                 'authorize_callback'
    # username="USERNAME",
    # password="PASSWORD"
)

In [20]:
import time
from tqdm.notebook import tqdm
topic_library = {
    'CryptoCurrency' : [ 'https://www.reddit.com/r/CryptoCurrency/comments/xvzivt/update_bitcoin_ceo_announces_075_transaction_fee/'
                  ],
    'Sports': [
        'https://www.reddit.com/r/DynastyFF/comments/xwmwtp/justin_fields_buy_low_or_completely_avoid_in_sf/',
        'https://www.reddit.com/r/fantasyfootball/comments/xwd5fk/kyle_pitts_gets_all_of_the_attention_for_te/', 
        ]
}
for topic, urls in tqdm(topic_library.items()):
  for url in tqdm(urls):
    submission = await reddit.submission(url=url)
    submission.comments.replace_more(limit=0) # flatten tree
    all_comments = []
    comments = submission.comments # all comments
    # comments = submission.comments.list() # all comments
    for top_level_comment in submission.comments:
      if isinstance(top_level_comment, MoreComments):
          continue
      all_comments.append(top_level_comment.body)
      with open('{}-reddit_author.txt'.format(topic), 'a') as f:
          f.write(submission.selftext + '\n')
      with open('{}-reddit_comments.txt'.format(topic), 'a') as f:
        for i, comment in enumerate(comments):
          f.write(comment.body + '\n')
    # time.sleep(1.5)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
#Gdrive access
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [22]:
import shutil
for topic, _ in tqdm(topic_library.items()):
  shutil.copy2('{}-reddit_author.txt'.format(topic), '/content/gdrive/MyDrive/Projects/Data/')
  shutil.copy2('{}-reddit_comments.txt'.format(topic), '/content/gdrive/MyDrive/Projects/Data/')

  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
import pandas as pd
reddit_df = pd.DataFrame(columns = ['text'])
for topic, _ in tqdm(topic_library.items()):
  with open('{}-reddit_comments.txt'.format(topic)) as f:
    lines = f.readlines()
    reddit_sub_df = pd.DataFrame(lines, columns = ['text'])
    reddit_sub_df = reddit_sub_df.sample(3000)
    reddit_df= reddit_df.append(reddit_sub_df).reset_index(drop=True)
# lines

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
reddit_df

,text
0,I swear like 95% of you don't know what comedy...
1,next headline would prolly be Bitcoin shifts f...
2,Next move: Bitcoin shifts from PoW to PoS\n
3,Still Bitcoin 100k eoy ???\n
4,Missed the comedy flair. Was wtf-ing all the w...
...,...
5995,Defenses are throwing suddenly weird alignment...
5996,I'd be interested to see what happens after th...
5997,\n
5998,\n


In [26]:
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words = 'english', max_df = 0.7)
tfidf_tokens = tfidf_vect.fit_transform(reddit_df['text'])
tfidf_vocab = tfidf_vect.get_feature_names_out()

In [27]:
print(tfidf_tokens[0])
tfidf_vocab[100]

  (0, 725)	0.4231760798913951
  (0, 222)	0.33495004550677693
  (0, 631)	0.3097971498498657
  (0, 331)	0.30599358614557187
  (0, 59)	0.46771372555027096
  (0, 658)	0.28567790511204033
  (0, 1127)	0.46771372555027096


'average'

In [28]:
lda = LatentDirichletAllocation(n_components = 10, doc_topic_prior=1)
lda.fit(tfidf_tokens)

LatentDirichletAllocation(doc_topic_prior=1)

In [29]:
import numpy as np 
topic_words = {}
n_top_words = 15
for topic, comp in enumerate(lda.components_):
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    # store the words most relevant to the topic
    topic_words[topic] = [tfidf_vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  league, tackle, shit, read, te, increase, kelce, just, weeks, andrews, op, kittle, need, drunk, bust
Topic: 1
  comedy, believe, don, people, lol, actually, good, know, like, dividends, better, dad, speak, totally, bitcoin
Topic: 2
  btc, says, delete, lie, blocker1, fantasy, ll, ready, directors, weird, works, bad, board, writing, wasted
Topic: 3
  kittle, pitts, week, real, wtf, way, starting, low, fee, great, days, considering, hyped, higbee, seriously
Topic: 4
  bitcoin, ceo, just, nouuuu, pos, pow, shifts, secretly, short, knows, run, isn, news, guy, members
Topic: 5
  drafted, draft, end, like, pitts, list, ceo, feel, interested, kittle, guess, leagues, games, sf, dogecoin
Topic: 6
  got, kittle, bitcoin, round, traded, perfect, like, slot, pick, george, game, ve, catch, points, needs
Topic: 7
  going, lmao, bitcoin, wonder, regarded, x200b, https, com, second, waivers, got, hold, bear, value, eoy
Topic: 8
  satoshi, really, sniffles, deleted, pitts, got, open, true, f

#Get wikipedia topics

In [59]:
documents2 = []

import re
regex = re.compile(r'<[^>]+>')

def remove_html(string):
    return regex.sub('', string)

def getContent2(url): 
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")
  for script in soup(["script", "style"]):
    script.extract()
  text = soup.get_text()
  documents2.append(remove_html(text))

url="https://en.wikipedia.org/wiki/United_States"
getContent2(url)
(documents2[0])[:500]
url="https://en.wikipedia.org/wiki/India"
getContent2(url)


In [60]:
#tfidf_vect = TfidfVectorizer(analyzer='word', stop_words = 'english')
df=tfidf_vect.fit_transform(documents2)
vocab = tfidf_vect.get_feature_names_out()

In [61]:
vocab

array(['00', '000', '0002', ...,
       '日本語napulitanoߒߞߏнохчийнnordfriisknorfuk', 'ꯟminangkabaumìng',
       '𐌲𐌿𐍄𐌹𐍃𐌺ग'], dtype=object)

In [62]:
lda.fit(df)

LatentDirichletAllocation(doc_topic_prior=1)

In [63]:
topic_words = {}
n_top_words = 20
for topic, comp in enumerate(lda.components_):
    # print(topic, comp)
    word_idx = np.argsort(comp)[::-1][:n_top_words] #argsort to get index, and [::-1] to sort in descending
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
    # break
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  highest, rico, puerto, spanish, pew, civil, rights, usa, alaska, house, bureau, health, public, great, department, service, income, senate, 1776, 1945
Topic: 1
  highest, rico, puerto, spanish, pew, civil, rights, usa, alaska, house, bureau, health, public, great, department, service, income, senate, 1776, 1945
Topic: 2
  highest, rico, puerto, spanish, pew, civil, rights, usa, alaska, house, bureau, health, public, great, department, service, income, senate, 1776, 1945
Topic: 3
  retrieved, states, united, american, 2011, new, war, 2020, america, 2022, 2015, january, june, americans, 2013, state, federal, york, september, march
Topic: 4
  india, isbn, 978, world, indian, original, pp, archived, july, press, history, university, october, south, 2014, pdf, 2021, 10, 2010, national
Topic: 5
  highest, rico, puerto, spanish, pew, civil, rights, usa, alaska, house, bureau, health, public, great, department, service, income, senate, 1776, 1945
Topic: 6
  highest, rico, puerto, sp